In [1]:
import numpy as np
import re
import pandas as pd

#load email from file
File = open("emailSample1.txt","r")
raw_email = File.read()

In [2]:
print(raw_email)

> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com




In [3]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

#load vocabulary
Dictionary = pd.read_csv('vocab.txt',delim_whitespace=True,header = None)

#create word_indices from raw email
    
def process_email(email):
    
    vocab_list = list(Dictionary[1])
    email_1 = list()
    word_indices = list()
    
    #remove header
    #index = email.index('\n\n')
    #email = email[index:]
    
    #lower case all words in raw email
    email = email.lower()
    
    #strip HTML TAGS
    email = re.sub("<[^<>]+>", " ", email)
    
    #Handle Numbers
    email = re.sub('[0-9]+','number',email)
    
    #Handle URL's
    email = re.sub('(http|https)://[^\s]*', 'httpaddr',email)
    
    #Handle Email Address
    email = re.sub('[^\s]+@[^\s]+', 'emailaddr',email)
    
    #Handle Dollar
    email = re.sub('[$]+', 'dollar',email)
    
    #Remove Punctuation
    email = re.split('[ \@\/\#\.\-\:\&\*\+\=\[\]\?\!\(\)\{\}\,\'\>\_\<\;\%\\n]',email)
    
    for token in email:
        if len(token) > 1:
            
            #stemming of word
            token = ps.stem(token)
            email_1.append(token)
            try:
                word_indices.append(Dictionary[0][vocab_list.index(token)]) 
            except ValueError:
                continue
    
   
    print(email_1)
    return email_1 , word_indices
    
proc_email , word_indices = process_email(raw_email)
len(word_indices)

['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani', 'visitor', 'you', 're', 'expect', 'thi', 'can', 'be', 'anywher', 'from', 'less', 'than', 'number', 'buck', 'month', 'to', 'coupl', 'of', 'dollarnumb', 'you', 'should', 'checkout', 'httpaddr', 'or', 'perhap', 'amazon', 'ecnumb', 'if', 'your', 'run', 'someth', 'big', 'to', 'unsubscrib', 'yourself', 'from', 'thi', 'mail', 'list', 'send', 'an', 'email', 'to', 'emailaddr']


53

In [63]:
#Feature Extraction
n = len(Dictionary)

def emailFeatures(word_indices,a):
    features = np.zeros((a,1))
    
    for index in word_indices:
        features[index - 1] = 1
    return features

feature_matrix = emailFeatures(word_indices,n)

In [5]:
#Train Linear SVM for Spam Classification

import scipy.io as si
from sklearn import svm

#load Training data from spamTrain.mat file
Training_data = si.loadmat('spamTrain.mat')

#load Cross-Validation Data
Cross_Val_data = si.loadmat('spamTest.mat')

C = .1

model = svm.SVC(C , kernel = 'linear')
model.fit(Training_data['X'] , Training_data['y'].ravel())

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
from sklearn import metrics #to calculate accuracy

def Accuracy(model,X,y):
    
    predicted = model.predict(X)
    accuracy = metrics.accuracy_score(y,predicted)
    return accuracy

print("Accuray in Training Data :" , Accuracy(model,Training_data['X'],Training_data['y']) * 100)
print("Accuray in Test Data :" , Accuracy(model,Cross_Val_data['Xtest'],Cross_Val_data['ytest']) * 100)

Accuray in Training Data : 99.825
Accuray in Test Data : 98.9


In [7]:
#Find Top Predictors for spam

coefficients = model.coef_[0]
weights = sorted(coefficients , reverse = True)
vocab_list = list(Dictionary[1])

for i in range(20):
    index = list(coefficients).index(weights[i])
    print(vocab_list[index] , weights[i])

our 0.500613736175
click 0.465916390689
remov 0.422869117061
guarante 0.383621601794
visit 0.367710398246
basenumb 0.345064097946
dollar 0.323632035796
will 0.269724106037
price 0.267297714618
pleas 0.2611688867
most 0.257298197952
nbsp 0.25394145516
lo 0.253466524314
ga 0.248296990456
hour 0.246404357832
al 0.237310668172
da 0.233261215232
se 0.23295496246
want 0.23194709266
dollarnumb 0.229639162845


In [8]:
#Try our own Examples

#load Test email from file
T_File = open("test.txt","r")
test_email = T_File.read()
print(test_email)

#processed our Test Email
proc_test_email , test_word_indices = process_email(test_email)

#create feature vector
test_feature_vector = emailFeatures(test_word_indices)

#predict
test_predict_val = model.predict(test_feature_vector.T)

print(test_predict_val)

From Steve_Burt@cursor-system.com  Thu Aug 22 12:46:39 2002
Return-Path: <Steve_Burt@cursor-system.com>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id BE12E43C34
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:46:38 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:46:38 +0100 (IST)
Received: from n20.grp.scd.yahoo.com (n20.grp.scd.yahoo.com
    [66.218.66.76]) by dogma.slashnull.org (8.11.6/8.11.6) with SMTP id
    g7MBkTZ05087 for <zzzz@example.com>; Thu, 22 Aug 2002 12:46:29 +0100
X-Egroups-Return: sentto-2242572-52726-1030016790-zzzz=example.com@returns.groups.yahoo.com
Received: from [66.218.67.196] by n20.grp.scd.yahoo.com with NNFMP;
    22 Aug 2002 11:46:30 -0000
X-Sender: steve.burt@cursor-system.com
X-Apparently-To: zzzzteana@yahoogroups.com
Received: (EGP: mail-8_1_0_1); 22 Aug 2002 11:46:

In [46]:
import os #to get names of each email from different folders
import random #to shuffle the items of a list

#contain names of Email and integer to tell Email is spam or not
Email_names = list()

#get non-spam Easy Email names
for filename in os.listdir('/home/shivam/email/20021010_easy_ham/easy_ham'):
    Email_names.append((filename,0))
    
#get non-spam Hard Email names
for filename in os.listdir('/home/shivam/email/20021010_hard_ham/hard_ham'):
    Email_names.append((filename,0))

#get Spam Email names
for filename in os.listdir('/home/shivam/email/spam'):
    Email_names.append((filename,1))

    
#shuffle the elements of list
random.shuffle(Email_names)

print(Email_names[3400])

('2011.9604a6dfecb37414590009d7c0c04f30', 0)


In [47]:
def proc_email(email):
    
    email_1 = list()
    
    #remove header
    
    email = email[email.find('\n\n'):]
    
    #lower case all words in raw email
    email = email.lower()
    
    #strip HTML TAGS
    email = re.sub("<[^<>]+>", " ", email)
    
    #Handle Numbers
    email = re.sub('[0-9]+','number',email)
    
    #Handle URL's
    email = re.sub('(http|https)://[^\s]*', 'httpaddr',email)
    
    #Handle Email Address
    email = re.sub('[^\s]+@[^\s]+', 'emailaddr',email)
    
    #Handle Dollar
    email = re.sub('[$]+', 'dollar',email)
    
    #Remove Punctuation
    email = re.split('[ \@\/\#\.\-\:\&\*\+\=\[\]\?\!\(\)\{\}\,\'\>\_\<\;\%\\n\\t]',email)
    
    for token in email:
        if len(token) > 1:
            
            #stemming of word
            token = ps.stem(token)
            email_1.append(token)
    
    return email_1

word_list = list()

#Create word_list
for i in range(len(Email_names)):
    T1_File = open('/home/shivam/email/main_data/' + Email_names[i][0] ,"r", errors='ignore')
    test_email = T1_File.read()
    word_list.append(proc_email(test_email))

In [48]:
from collections import defaultdict #to initializze every key's value 0

count_word = defaultdict(lambda: 0)

#count occurence of a word in given number of Emails
for i in range(len(word_list)):
    for word in word_list[i]:
        count_word[word] = count_word[word] + 1

In [49]:
from collections import Counter  #to get most common words

#Get most occured words from Emails and stored in vocab_list
new_vocab_list = list(dict(Counter(count_word).most_common(2500)).keys())
new_vocab_list

#Create word_indices for Each Email
new_word_indices = list()

for i in range(len(word_list)):
    temp = list()
    for word in word_list[i]:
            try:
                temp.append(new_vocab_list.index(word) + 1) 
            except ValueError:
                continue
    new_word_indices.append(temp)

In [65]:
#create feature vector for each email and also create Y(actual result)

N = len(new_vocab_list)
M = len(Email_names)

new_feature_vector = np.zeros((M,N))
    
for i in range(len(new_word_indices)):
    for index in new_word_indices[i]:
        new_feature_vector[i][index - 1] = 1

#create Y
Y = np.zeros((M,1))
for i ,data in enumerate(Email_names):
    Y[i] = data[1]

In [51]:
#split data as Training and Testing Data
from sklearn.cross_validation import train_test_split #libraray to split data

X_train_new , X_test_new , Y_train_new , Y_test_new = train_test_split(new_feature_vector , Y ,test_size = .3)


In [52]:
#Train Linear SVM for Spam Classification

import scipy.io as si
from sklearn import svm

C = .1

model_new = svm.SVC(C , kernel = 'linear')
model_new.fit(X_train_new , Y_train_new.ravel())

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [53]:
from sklearn import metrics #to calculate accuracy

print("Accuray in Training Data :" , Accuracy(model_new,X_train_new,Y_train_new) * 100)
print("Accuray in Test Data :" , Accuracy(model_new,X_test_new,Y_test_new) * 100)

Accuray in Training Data : 99.96243426
Accuray in Test Data : 98.4224364592


In [71]:
#Try our own Examples

#load Test email from file
T_File = open("test.txt","r")
test_email = T_File.read()
print(test_email)

#processed our Test Email
proc_test_email  = proc_email(test_email)

print(proc_test_email)

test_word_indices = list()
for word in proc_test_email:
            try:
                test_word_indices.append(new_vocab_list.index(word) + 1) 
            except ValueError:
                continue

print(test_word_indices)

#create feature vector
test_feature_vector = emailFeatures(test_word_indices,N)

print(test_feature_vector.shape)

#predict
test_predict_val = model_new.predict(test_feature_vector.T)

print(test_predict_val)



Dear A,

There is a saying that it takes 10,000 hours of doing something to master it. So, to master programming, it might take you 10,000 hours of being actively coding or thinking about coding. That translates to a consistent effort spread out over a number of years. 

With more than 1,00,000 people already learning to become a master in coding, these courses will give you a head-start to your journey in coding. All the courses have been designed by IIT Alumnis are are updated with the latest trends.

These courses are designed so they can be completed In-Semester and only require 3-5 hours/week.

Key Elements of the Courses :

1. Great learning of Industrial Skills from Experts
2. One to One Discussion with Experts
3. Assignments related to subject area
4. Live Projects of Industries with Internship

8 Most important coding certification courses for all:

1. Minor Degree in Programming (Save 40%)- Click Here

2. Python Programming Certification - Click Here

3. Machine Learning Ce